In [28]:
from transformers import BertTokenizer
import numpy as np
from tqdm import tqdm
import tensorflow as tf

In [7]:
# processed_sentences = np.load('processed_sentences.npy', allow_pickle=True)
# processed_sentences = np.load(
#     'processed_sentences_no_conct.npy', allow_pickle=True)
# processed_sentences = np.load('processed_sentences_no_conct.npy', allow_pickle=True)
# t=np.load('entities_to_words_not_processed.npy', allow_pickle=True)
dev_processed_sentences = np.load('dev_processed_sentences_no_conct.npy', allow_pickle=True)
dev_t=np.load('dev_entities_to_words_not_processed.npy', allow_pickle=True)


In [8]:
t[0]

['O', 'O', 'O', 'NUMBER', 'SIZE', 'TOPPING', 'TOPPING', 'TOPPING']

In [9]:
processed_sentences[0]

np.str_('can i have a large bbq pulled pork')

In [3]:
processed_sentences[5].tolist().split()

['i',
 'need',
 'to',
 'order',
 'one',
 'large',
 'pizza',
 'with',
 'ham',
 'bacon',
 'onions',
 'and',
 'black',
 'olives',
 'one',
 'medium',
 'pizza',
 'with',
 'sausage',
 'and',
 'onions',
 'and',
 'six',
 'large',
 'cokes']

In [10]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [29]:
def prepare_data(sentences, labels, tokenizer, max_length):
    input_ids = []
    attention_masks = []

    for sentence in tqdm(sentences, desc="Tokenizing sentences"):
        encoding = tokenizer(sentence,
                             truncation=True,
                             padding='max_length',
                             max_length=max_length,
                             return_tensors="tf")  # Use "tf" to return TensorFlow tensors

        input_ids.append(encoding["input_ids"])  # TensorFlow tensor
        attention_masks.append(encoding["attention_mask"])  # TensorFlow tensor
    
    # Convert the list of tensors into a single tensor (batch_size, sequence_length)
    input_ids = tf.concat(input_ids, axis=0)
    attention_masks = tf.concat(attention_masks, axis=0)

    # Convert labels to a TensorFlow tensor as well
    labels = tf.convert_to_tensor(labels)

    return input_ids, attention_masks, labels

In [4]:
entities=np.load('full_negate_entities.npy', allow_pickle=True)
entities_id = {e.item(): i+1 for i, e in enumerate(entities)}
entities_id['0']=0
entities_id['O']=0

In [5]:
def padding_labels(labels, max_len):
    for i in range(len(labels)):
        if len(labels[i]) < max_len:
            labels[i] = labels[i] + ['0'] * (max_len - len(labels[i]))
        else:
            labels[i] = labels[i][:max_len]
    return labels

In [31]:
labels=[['0']+[entities_id[i] for i in tq]+['0'] for tq in t]
padded_labels = padding_labels(labels, 30)
padded_labels=np.array(padded_labels, dtype=np.int32)

In [8]:
labels_dev=[['0']+[entities_id[i] for i in tq]+['0'] for tq in dev_t]
padded_labels_dev = padding_labels(labels_dev, 30)
padded_labels_dev=np.array(padded_labels_dev, dtype=np.int32)

In [33]:
padded_labels[0]

array([ 0,  0,  0,  0, 15,  2,  7,  7,  7,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [44]:
input_ids_train, attention_masks_train, padded_labels_train = prepare_data(processed_sentences, padded_labels, tokenizer, 30)

Tokenizing sentences: 100%|██████████| 2456446/2456446 [09:59<00:00, 4099.97it/s]


In [45]:
print(input_ids_train.shape)
print(attention_masks_train.shape)
print(padded_labels_train.shape)

(2456446, 30)
(2456446, 30)
(2456446, 30)


In [46]:
np.save('input_ids_train.npy', input_ids_train)
np.save('attention_masks_train.npy', attention_masks_train)
np.save('padded_labels_train.npy', padded_labels_train)


In [30]:
input_ids_dev, attention_masks_dev, padded_labels_dev = prepare_data(dev_processed_sentences, padded_labels_dev, tokenizer, 30)

Tokenizing sentences: 100%|██████████| 348/348 [00:00<00:00, 2615.07it/s]


In [36]:
print(input_ids_dev.shape)
print(type(input_ids_dev[0]))
print(attention_masks_dev.shape)
print(type(attention_masks_dev[0]))
print(padded_labels_dev.shape)
print(type(padded_labels_dev[0]))

(348, 30)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(348, 30)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(348, 30)
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [37]:
np.save('input_ids_dev.npy', input_ids_dev)
np.save('attention_masks_dev.npy', attention_masks_dev)
np.save('padded_labels_dev.npy', padded_labels_dev)

In [39]:
# input_ids_train[0]
# attention_masks_train[0]
# padded_labels_train[0]

array([101, 2064, 1045, 2031, 1037, 2312, 22861, 4160, 2766, 15960, 102,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=object)

In [5]:
tokenized_sentences = [
    tokenizer.tokenize(sentence) for sentence in tqdm(processed_sentences, desc="Tokeninzing Sentences")
]

Tokeninzing Sentences:   0%|          | 0/348 [00:00<?, ?it/s]

Tokeninzing Sentences: 100%|██████████| 348/348 [00:00<00:00, 6005.74it/s]


In [6]:
tokenized_sentences_numpy= np.array(tokenized_sentences, dtype=object)

In [7]:
# np.save('tokenized_sentences.npy', tokenized_sentences_numpy)
# np.save('tokenized_sentences_no_conct.npy', tokenized_sentences_numpy)
np.save('dev_tokenized_sentences_no_conct.npy', tokenized_sentences_numpy)
